#EduWeaver

EduWeaver uses ChatGPT API to make a LiaScript course. EduWeaver can be considered an AutoCourse maker. 

- Specify a topic and a few other parameters (i.e. sub-sections to nclude and exclude)
- Creates a list of sub sections (or chapters)
- Generates the content
- Generates appropriate activities eg MCQ's
- Adds simplified sections to explain complex things better
- Includes external links or readings
- Includes code snippets/examples for programming related topics
- Saves as a Markdown file for valid [LiaScript](https://liascript.github.io/)

Coming soon:

- Additional interactive learning activities
- Other export formats eg HTML and SCORM

## Setup

Enter a few details about the course and your OpenAI API key. You can create a new API key by going to your OpenAI Account [https://platform.openai.com/account/api-keys].

In [16]:
#@title Course Generation Settings
topic = 'Writing and Evaluating Lesson Plans'  #@param {type: "string"}
teaching_method = 'Explain concepts is a simple manner and include relevant learning theories and innovative learning activities where possible.'  #@param {type: "string"}
no_sub_topics = 6  #@param {type: "slider", min: 2, max: 10}
sub_topics_to_include = "Lesson plans for remote classes using using Zoom or Microsoft Teams" #@param {type: "string"}
sub_topics_to_not_include = "Learning styles" #@param {type: "string"}
audience = 'First Year University Students Enrolled to become a Teacher'   #@param {type: "string"}
no_quiz_questions_in_sub_topic = 5 #@param {type: "slider", min: 1, max: 6}
narrator_language = "Australian Female" #@param ["UK English Female", "US English Female", "Australian Female", "UK English Male", "US English Male", "Australian Male"]
output_filename = 'LessonPlans_Course.md'  #@param {type: "string"}

#@markdown ---


In [ ]:
# Set your environment variables
# Don't include quotes around the key
%env OPENAI_API_KEY=

In [18]:
# Install required libraries
!pip -q install openai langchain

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# Get Sections in Topic

# First extract the additional specified sub-sections
list_of_additional_sub_sections = sub_topics_to_include.split(';') 
sub_topics_to_include_text = " "
if len(list_of_additional_sub_sections) > 0:
  sub_topics_to_include_text = "In the sub sections also please include these: " + sub_topics_to_include

list_of_sub_topics_not_to_incude = sub_topics_to_not_include.split(';') 
sub_topics_to_not_include_text = " "
if len(list_of_sub_topics_not_to_incude) > 0:
  sub_topics_to_not_include_text = "Please don't include content on this (this is very important): " + sub_topics_to_not_include


if teaching_method!="":
  teaching_method_to_include_text = "You this teaching and writing style: " + teaching_method

chat = ChatOpenAI(temperature=0.7)

act_as_ld_template='''
                      Please act as a subject matter expert on the topic of {topic} that also has learning designer skills and knowledge of markdown and json. 
                      You know how to design online modules. You can break up the content into chunks, explain concepts so that can easily be understood, write quizzes to test knowledge and write creative learning activities for students. 
                   '''
system_message_prompt = SystemMessagePromptTemplate.from_template(act_as_ld_template)
ask_for_sections_template='''
                              Could you please come up with {no_sub_topics} sections for an online module on the topic of {topic} for {audience} in json format?
                              {sub_topics_to_include_text}
                              {sub_topics_to_not_include_text}
                              Here is an example of the format you need to return:
                              [{{"section_name": "The name of the section 1", "section_description": "The descriptions of the section 1"}}]
                              Please don't return any intro text before or after the json.
                          '''
human_message_prompt = HumanMessagePromptTemplate.from_template(ask_for_sections_template)


chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

print("- Getting the sub sections to include.")

# get a chat completion from the formatted messages
chat_response = chat(chat_prompt.format_prompt(topic=topic, no_sub_topics=no_sub_topics, audience=audience, sub_topics_to_include_text=sub_topics_to_include_text, sub_topics_to_not_include_text=sub_topics_to_not_include_text).to_messages())

# convert sections json to dict

import json

sections_dict = json.loads(chat_response.content)

# Get content for each sub section

print("- Getting content for each sub section.")

course_content = {}

for section in sections_dict:
  section_name = section['section_name']
  section_description = section['section_description']
  ask_for_section_details_template='''
                              Could you please write the content for {section_name} that will be understood by {audience}?
                              The description of the topic that you must write is {section_description}.
                              {teaching_method_to_include_text}
                              {sub_topics_to_not_include_text}
                              Please don't output the section name but do include an introductory paragraph.
                              If the section is on programming or coding please include code examples within backticks and specify the language and filename eg:
                              ``` js     +Filename.js
                                  let hi = "Hello World"
                              ```
                              If the section needs math just place the Latex for the math equations between $ signs eg $ \frac{{a}}{{\sum{{b+i}}}} $ 
                              You can include sub sections. Please place the sub section headings within ** and ** followed by \n (eg **Goals** \n ) as this is the markdown for sub sections at heading level 3 and a newline is required after. 
                              For difficult concept please include a section also "**Additional Links**" with links to Wikipedia and please make sure the proper markdown is used for links. Only include links to wikipedia that exist, don't include links to other sites (this is very important).
                          '''
  human_message_prompt = HumanMessagePromptTemplate.from_template(ask_for_section_details_template  )

  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


  # get a chat completion from the formatted messages
  chat_response = chat(chat_prompt.format_prompt(topic=topic, section_name=section_name, section_description=section_description, audience=audience, teaching_method_to_include_text=teaching_method_to_include_text, sub_topics_to_not_include_text=sub_topics_to_not_include_text).to_messages())

  course_content[section_name] = chat_response.content

# Get creative projects each sub section

print("- Getting creative projects for each sub section.")

project_content = {}

for section in sections_dict:
  section_name = section['section_name']
  section_description = section['section_description']
  ask_for_section_project_template='''
                              Could you please come up with a few creative project ideas for {section_name} that will be appropriate for {audience}? 
                              The description of the topic that you must write creative project ideas for is {section_description}.
                              Be creative and the project should assist students in learning higher level skills that can't be obtained by doing multiple choice tests. 
                              The high level project ideas should develop levels from Blooms taxonomy including Knowledge, Comprehension, Application, Analysis, Synthesis, and Evaluation. 
                              Please don't output the section name or the type of level used from Blooms taxonomy but do include an introductory paragraph.
                              If the project for the section is on programming or coding please include code examples for the project within backticks and specify the language and filename eg:
                              ``` js     +Filename.js
                                  let hi = "Hello World"
                              ```
                              If the project for the section needs maths please place the Latex for the math equations between $ signs eg $ \frac{{a}}{{\sum{{b+i}}}} $
                          '''
  human_message_prompt = HumanMessagePromptTemplate.from_template(ask_for_section_project_template  )

  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


  # get a chat completion from the formatted messages
  chat_response = chat(chat_prompt.format_prompt(topic=topic, section_name=section_name, section_description=section_description, audience=audience).to_messages())

  project_content[section_name] = chat_response.content


# Get Quiz Questions for each sub section

print("- Creating MCQ's for each section.")

course_assessment = {}

for section in sections_dict:
  section_name = section['section_name']
  section_description = section['section_description']
  ask_for_section_assessment_template='''
                              Could you please write quiz questions for the content in {section_name} that will be appropriate for {audience}?
                              The description of the topic that you must write quiz questions for is {section_description}.
                              Please write {no_quiz_questions_in_sub_topic} questions in the json format that is given in the example below. 
                              [
                                {{"question": "Question 1",
                                 "question_type": "single_option",
                                 "options": [{{"option_name": "Option 1", "correct": "false"}}, {{"option_name": "Option 2", "correct": "true"}}]
                                }},
                                {{"question": "Question 2",
                                 "question_type": "multiple_options",
                                 "options": [{{"option_name": "Option 1", "correct": "false"}}, {{"option_name": "Option 2", "correct": "true"}}, {{"option_name": "Option 3", "correct": "true"}} the]
                                }}
                              ]
                              The questions must be returned in the same json format as what is given above. The incorrect answers should not be obvious or easy.
                          '''
  human_message_prompt = HumanMessagePromptTemplate.from_template(ask_for_section_assessment_template  )

  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


  # get a chat completion from the formatted messages
  chat_response = chat(chat_prompt.format_prompt(topic=topic, no_quiz_questions_in_sub_topic=no_quiz_questions_in_sub_topic, section_name=section_name, section_description=section_description, audience=audience).to_messages())

  course_assessment[section_name] = chat_response.content



- Getting the sub sections to include.
- Getting content for each sub section.
- Getting creative projects for each sub section.
- Creating MCQ's for each section.


In [21]:
# Convert Quiz Json to LiaScript Markdown
print("- Convert MCQ Json to Liascript Markdown.")

course_assessment_markdown = {}

for section in sections_dict:
  section_name = section['section_name']
  quiz_json = course_assessment[section_name]
  try:
    quiz_dict = json.loads(quiz_json)
    
    question_markdown = ""
    for question in quiz_dict:
      question_title = question['question']
      question_type = question['question_type']
      if (question_type == "single_option"):
        open_brace = '('
        close_brace = ')'
      else:
        open_brace = '['
        close_brace = ']'      
      question_markdown += question_title + '\n\n'
      options = question['options']
      for option in options:
        option_name = option["option_name"]
        if option['correct'] == "true":
          marker = 'X'
        else:
          marker = ' '
        question_markdown += f'''    [{open_brace}{marker}{close_brace}] {option_name} \n'''
      question_markdown += '\n'
      course_assessment_markdown[section_name] = question_markdown
  except json.decoder.JSONDecodeError:
	  print("String could not be converted to JSON")

# Collate course and force download of the Markdown file

print("- Collating course content.")

course_markdown = ""

course_markdown += f"""
<!--

author:   EduWeaver - An AutoCourse Creator Using ChatGPT
email:    nobody@nowhere.com
version:  0.0.2
language: en
narrator: {narrator_language}

logo:     https://liascript.github.io/img/bg-showcase-1.jpg

comment:  Eduweaver generates course content using chatGPT and outputs in Liascript Markdown

-->
"""

course_markdown += f'''# {topic}\n'''

# Make the TOC
toc = "In this course the following content will be covered: \n\n"
for section in sections_dict:
  section_name = section['section_name']
  section_description = section['section_description'] 
  toc += f'''- {section_name} \n {section_description} \n\n'''

course_markdown += """
> This course is completely generated by AutoCourse (using ChatGPT) in Liascript Markdown format.
> Please verify the content before publishing the course. \n \n
"""

course_markdown += toc

for section in sections_dict:
  section_name = section['section_name']
  section_description = section['section_description']
  section_content = course_content[section_name]
  if section_name in course_assessment_markdown:
    section_assessment = course_assessment_markdown[section_name]
  else:
    section_assessment = None
  project_ideas = project_content[section_name]
  course_markdown += f'''## {section_name}\n'''
  course_markdown += f'''{section_description}\n'''
  course_markdown += f'''{section_content}\n\n'''
  if section_assessment!=None:
    course_markdown += f'''### Quiz Questions \n\n'''
    course_markdown += f'''{section_assessment}\n'''
  course_markdown += f'''### Project Ideas \n\n'''
  course_markdown += f'''{project_ideas}\n\n'''
  
with open(output_filename, 'w') as writefile:
    writefile.write(course_markdown)

print("- Forcing file download.")
from google.colab import files
files.download(output_filename)

print("- If the file does not download, click on the Folder on the left and you will be able to download manually.")

- Convert MCQ Json to Liascript Markdown.
- Collating course content.
- Forcing file download.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- If the file does not download, click on the Folder on the left and you will be able to download manually.


## What to do next?

- Place the markdown (.md) file on github and then view using the [Liascript Viewer](https://liascript.github.io/)
OR
- Copy the markdown content into the [Liascript LiveEditor](https://liascript.github.io/LiveEditor/) for editing and previewing.